##Libraries and environment preparation

In [1]:
#Colab Environment Check for GPU and RAM
!nvidia-smi

Wed Mar 30 18:04:40 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
#Install essential packages
%%capture
!pip install datasets transformers rouge-score nltk wandb
!apt install git-lfs/

In [3]:
# Make sure your version of Transformers is at least 4.11.0 
# to run the following code correctly:
import datasets
import transformers
print(transformers.__version__)

4.17.0


In [4]:
from transformers import AutoTokenizer    
# Huggingface Automodel class
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model_checkpoint = "facebook/bart-base"

In [5]:
# Import Wandb 
import os
import wandb
API_KEY = '39991c538626bee25c64d4f8a4c3403dd635537c'
os.environ["WANDB_API_KEY"] = API_KEY

##Load the dataset

In [6]:
# import dataset
raw_datasets = datasets.load_dataset("cnn_dailymail", "3.0.0")

Extracting data files:   0%|          | 0/5 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

Dataset cnn_dailymail downloaded and prepared to /root/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/3cb851bf7cf5826e45d49db2863f627cba583cbc32342df7349dfe6c38060234. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})

##Preprocess the data

In [8]:
# Import tokenizer from model checkpoint
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/1.68k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

In [9]:
# tokenlize inputs into map

max_input_length = 512
max_target_length = 128

def preprocess_function(examples):
    inputs = [doc for doc in examples["article"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["highlights"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [10]:
tokenized_train = raw_datasets["train"].map(preprocess_function, 
                                      batched=True,
                                      remove_columns=["article", "highlights", "id"])

  0%|          | 0/288 [00:00<?, ?ba/s]

In [11]:
tokenized_val = raw_datasets["validation"].map(preprocess_function, 
                                      batched=True,
                                      remove_columns=["article", "highlights", "id"])

  0%|          | 0/14 [00:00<?, ?ba/s]

In [12]:
tokenized_train

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 287113
})

##Fine-tuning the model

In [13]:
# Import tokenizer from model checkpoint and print detail
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

# data collator: pad the inputs and labels during each batch to save space
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

Downloading:   0%|          | 0.00/532M [00:00<?, ?B/s]

In [14]:
# Define traing args, batch size and epoch
# batch size max 8 for input length 1024 on Colab Pro

batch_size = 8
epochs = 1
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-cnn",
    evaluation_strategy="epoch",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    save_strategy = "epoch",
    save_total_limit=3,
    num_train_epochs=epochs,
    predict_with_generate=True,
    fp16=True,
    #report_to="wandb",
)

In [15]:
import nltk
import numpy as np
nltk.download('punkt')

metric = datasets.load_metric("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [16]:
# Pass into the trainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Using amp half precision backend


In [17]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 287113
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 35890
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Currently logged in as: shusunny (use `wandb login --relogin` to force relogin)


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,1.916400,1.699106,24.460200,12.266200,20.382500,23.063100,20.000000


***** Running Evaluation *****
  Num examples = 13368
  Batch size = 8
Saving model checkpoint to bart-base-finetuned-cnn/checkpoint-35890
Configuration saved in bart-base-finetuned-cnn/checkpoint-35890/config.json
Model weights saved in bart-base-finetuned-cnn/checkpoint-35890/pytorch_model.bin
tokenizer config file saved in bart-base-finetuned-cnn/checkpoint-35890/tokenizer_config.json
Special tokens file saved in bart-base-finetuned-cnn/checkpoint-35890/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=35890, training_loss=2.0273490967409096, metrics={'train_runtime': 15428.8208, 'train_samples_per_second': 18.609, 'train_steps_per_second': 2.326, 'total_flos': 8.75315747045376e+16, 'train_loss': 2.0273490967409096, 'epoch': 1.0})

In [18]:
wandb.finish()

eval/gen_len,▁
eval/loss,▁
eval/rouge1,▁
eval/rouge2,▁
eval/rougeL,▁
eval/rougeLsum,▁
eval/runtime,▁
eval/samples_per_second,▁
eval/steps_per_second,▁
train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
train/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████


In [19]:
!ls -lh

total 16K
drwxr-xr-x 4 root root 4.0K Mar 30 22:36 bart-base-finetuned-cnn
drwx------ 5 root root 4.0K Mar 30 18:04 drive
drwxr-xr-x 1 root root 4.0K Mar 23 14:22 sample_data
drwxr-xr-x 3 root root 4.0K Mar 30 18:19 wandb


In [20]:
!zip -r bart-base-finetuned-cnn.zip /content/bart-base-finetuned-cnn/checkpoint-35890/

  adding: content/bart-base-finetuned-cnn/checkpoint-35890/ (stored 0%)
  adding: content/bart-base-finetuned-cnn/checkpoint-35890/scaler.pt (deflated 55%)
  adding: content/bart-base-finetuned-cnn/checkpoint-35890/merges.txt (deflated 53%)
  adding: content/bart-base-finetuned-cnn/checkpoint-35890/scheduler.pt (deflated 49%)
  adding: content/bart-base-finetuned-cnn/checkpoint-35890/tokenizer_config.json (deflated 46%)
  adding: content/bart-base-finetuned-cnn/checkpoint-35890/optimizer.pt (deflated 9%)
  adding: content/bart-base-finetuned-cnn/checkpoint-35890/config.json (deflated 64%)
  adding: content/bart-base-finetuned-cnn/checkpoint-35890/pytorch_model.bin (deflated 8%)
  adding: content/bart-base-finetuned-cnn/checkpoint-35890/vocab.json (deflated 59%)
  adding: content/bart-base-finetuned-cnn/checkpoint-35890/tokenizer.json (deflated 72%)
  adding: content/bart-base-finetuned-cnn/checkpoint-35890/special_tokens_map.json (deflated 50%)
  adding: content/bart-base-finetuned-cnn

In [21]:
!cp bart-base-finetuned-cnn.zip '/content/drive/My Drive/weights/'